# Sheep Affymetrix chip aligned
I've aligned with megablast the affymetrix sheep chip probeset to the oldest `OAR4` assembly. I want to compare my results with result I get from dbSNP

In [1]:
from collections import defaultdict

from src.features.smarterdb import global_connection, VariantSheep
from src.data.common import AssemblyConf

import pandas as pd

In [2]:
conn = global_connection()
dbSNP152 = AssemblyConf(version="Oar_v4.0", imported_from="dbSNP152")

First, get my data aligned and set `snp_name` as a index:

In [3]:
results = pd.read_csv("Axiom_Ovi_Can.na35.r3.a3.annot.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.csv")
results.set_index("snp_name", inplace=True)
results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt
snp_name,,,,,,,,,
Affx-293815543,10,86367023,C/T,T/C,T/C,BOT,forward,T,C
Affx-139979198,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN
Affx-139969918,0,0,NaN,C/G,NaN,TOP,NaN,NaN,NaN
Affx-139932950,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN
Affx-139939859,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN


Next load errors: If I can't place a SNP on a chromosome, I will have no position in results table and I will have a reason in this table:

In [4]:
errors = pd.read_csv("Axiom_Ovi_Can.na35.r3.a3.annot.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.err")
errors.set_index("snp_name", inplace=True)
errors.head()

,illumina,illumina_strand,reason
snp_name,,,
Affx-139979198,T/G,BOT,No valid alignments after filtering
Affx-139969918,C/G,TOP,No valid alignments after filtering
Affx-139932950,T/C,BOT,No valid alignments after filtering
Affx-139939859,A/G,TOP,No valid alignments after filtering
Affx-139991202,A/G,TOP,No valid alignments after filtering


Ok, now get my Sheep variants and focus on *NCBI* data: I could have more variants than *NCBI* if there are probes more recent than dbSNP152:

In [5]:
ncbi_variants = VariantSheep.objects.filter(chip_name="AffymetrixAxiomOviCan", locations__match=dbSNP152._asdict(), rs_id__exists=True)
ncbi_variants.count()

39105

Ok now extract dbSNP locations from my `ncbi_variants`:

In [6]:
tmp = defaultdict(list)

for variant in ncbi_variants:
    location = variant.get_location(**dbSNP152._asdict())
    tmp["snp_name"].append(variant.affy_snp_id)
    tmp["rs_id"].append(",".join(variant.rs_id))
    tmp["ncbi_chrom"].append(location.chrom)
    tmp["ncbi_position"].append(location.position)
    
    
ncbi_locations = pd.DataFrame.from_dict(tmp)
ncbi_locations.set_index('snp_name', inplace=True)
ncbi_locations.head()

,rs_id,ncbi_chrom,ncbi_position
snp_name,,,
Affx-256854517,rs10721113,18,64294536
Affx-122852950,"rs406297509,rs1087899539",16,68777502
Affx-122806470,"rs402039066,rs1093088087",5,34727924
Affx-122839502,rs119102699,1,103285485
Affx-122821645,rs159412897,1,121010442


Ok, merge these data in a new dataframe. Get rid of SNPs not in *NCBI*, mind that since `ncbi_position` could have *NA* values, need to be converted as *integer*:

In [7]:
tmp = results.merge(ncbi_locations, how="left", on="snp_name")
ncbi_results = tmp[tmp['ncbi_chrom'].isna() == False].astype({'ncbi_position':'int'})
ncbi_results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
Affx-122847494,3,30880580,A/G,A/G,A/G,TOP,forward,A,G,rs424489686,3,30880580
Affx-122829181,2,219365951,A/G,A/G,A/G,TOP,forward,G,A,rs401909860,2,219365951
Affx-122816720,1,120533735,A/G,A/G,A/G,TOP,forward,A,G,rs398687222,1,120533735
Affx-122808678,16,22212130,C/T,T/C,T/C,BOT,forward,C,T,rs415806402,16,22212130
Affx-122814061,1,4556384,C/T,T/C,T/C,BOT,forward,C,T,rs55630584,1,4556384


Ok focus on the differences between my alignment and NCBI:

In [8]:
differences = ncbi_results.query("chrom != ncbi_chrom | position != ncbi_position")
differences.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
Affx-122859207,21,40958664,A/C,A/C,A/C,TOP,forward,A,C,rs418656823,21,40958663
Affx-122859153,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs428412553,2,6796820
Affx-122859040,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409528638,6,111790031
Affx-122858997,10,23088577,C/T,T/C,T/C,BOT,forward,C,T,rs412459161,10,23088576
Affx-122858615,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN,rs415678531,3,41459193


In [9]:
differences.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2449 entries, Affx-122859207 to Affx-122805472
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chrom             2449 non-null   object
 1   position          2449 non-null   int64 
 2   alleles           360 non-null    object
 3   illumina          2449 non-null   object
 4   illumina_forward  360 non-null    object
 5   illumina_strand   2449 non-null   object
 6   strand            360 non-null    object
 7   ref               360 non-null    object
 8   alt               360 non-null    object
 9   rs_id             2449 non-null   object
 10  ncbi_chrom        2449 non-null   object
 11  ncbi_position     2449 non-null   int64 
dtypes: int64(2), object(10)
memory usage: 248.7+ KB


I have ~2500 differences between NCBI, let's focus on different chromosome types:

In [10]:
differences["chrom"].value_counts()

0                 2089
2                   41
1                   38
3                   36
6                   22
4                   18
8                   17
7                   16
18                  15
5                   15
13                  14
14                  11
16                  11
19                  11
11                  10
17                  10
26                   9
12                   8
10                   8
15                   8
20                   7
9                    6
X                    6
25                   6
23                   5
22                   5
24                   3
21                   3
AMGL02043384.1       1
Name: chrom, dtype: int64

Ok, tell me how many SNPs I can't place, while *NCBI* can:

In [11]:
not_placed = differences[differences["chrom"] == '0']
print(f"There are {not_placed.shape[0]} SNPs that I can't map to genome")
not_placed.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 2089 SNPs that I can't map to genome


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
Affx-122821573,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,"rs419029666,rs407077541",4,71841104,Allele doesn't match to reference
Affx-122828099,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs409901274,24,33324073,Allele doesn't match to reference
Affx-122828089,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs402974360,3,16674065,Allele doesn't match to reference
Affx-122828089,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs402974360,3,16674065,Allele doesn't match to reference
Affx-122849788,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs424325727,2,172609389,Allele doesn't match to reference
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Affx-122843237,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs424022354,4,45287904,Too many alignments after filtering
Affx-122843237,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs424022354,4,45287904,Too many alignments after filtering
Affx-122818100,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs429219542,1,179238938,Too many alignments after filtering


Well, there are a lot of SNPs I cannot match. Group them by reason:

In [12]:
not_placed.merge(errors["reason"], how="left", on="snp_name")["reason"].value_counts()

No valid alignments after filtering    2268
Allele doesn't match to reference        79
Too many alignments after filtering      64
Name: reason, dtype: int64

In [13]:
tmp = not_placed.merge(errors["reason"], how="left", on="snp_name")
tmp[tmp["reason"] == "No valid alignments after filtering"]

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
Affx-122859153,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs428412553,2,6796820,No valid alignments after filtering
Affx-122859040,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409528638,6,111790031,No valid alignments after filtering
Affx-122858615,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN,rs415678531,3,41459193,No valid alignments after filtering
Affx-122858602,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs429068120,23,2448098,No valid alignments after filtering
Affx-122858585,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs408135675,26,31305174,No valid alignments after filtering
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Affx-122807329,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs417861422,1,106450244,No valid alignments after filtering
Affx-122807016,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN,rs416491353,2,67161544,No valid alignments after filtering
Affx-122807055,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs419002996,26,33849475,No valid alignments after filtering


Are then any *SNP* which I map to a different position than ncbi??

In [14]:
different = differences.query("chrom != '0' and ncbi_chrom != '0'")
print(f"There are {different.shape[0]} SNPs that I can map to a different position")
different.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 357 SNPs that I can map to a different position


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
Affx-122859207,21,40958664,A/C,A/C,A/C,TOP,forward,A,C,rs418656823,21,40958663,NaN
Affx-122858997,10,23088577,C/T,T/C,T/C,BOT,forward,C,T,rs412459161,10,23088576,NaN
Affx-122858454,2,97033699,A/G,A/G,A/G,TOP,forward,A,G,rs428349851,2,97033698,NaN
Affx-122858889,1,256186052,G/T,T/G,T/G,BOT,forward,T,G,rs429707229,1,256186051,NaN
Affx-122858956,3,46907236,C/T,T/C,T/C,BOT,forward,C,T,rs411322848,3,46907235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Affx-122808368,13,20786012,C/T,T/C,T/C,BOT,forward,C,T,rs400600156,13,20786011,NaN
Affx-122807919,20,37359516,C/T,T/C,T/C,BOT,forward,C,T,rs424386597,20,37359515,NaN
Affx-122807224,2,232354107,C/T,T/C,T/C,BOT,forward,T,C,rs55630574,2,232354106,NaN
